Define a function to run "Strategy 1" on a given maze. This strategy identifies the shortest path from the start to the goal location in the given maze one time at the start of the function. The agent is then moved along this path and the fire is advanced every time after the agent moves. If the agent reaches the goal, True is returned. Otherwise, False is returned.

In [ ]:
from maze_common import *

def first_Strategy(maze):
    m = maze
    path = None
    # calculate the shortest path after the instantiation of the maze (i.e obstacles set and fire started)
    result = shortestPathSearch(m)
    if result != None:
        path = result[0]
    if path == None:
        return False
    # using the initial path generated, iterate through the path, checking if one of the cells along the path catches fire
    for p in path:
        checki = p[0] 
        checkj = p[1]
        #print(checki)
        if maze.board[(checki,checkj)] == 2: # if the current cell is fire, return False
            return False
        elif checki == maze.dim - 1 and checkj == maze.dim - 1: # if goal, return True
            return True
        else:
            m.iterateFire() # iterate the fire if current cell is not goal or if agent is still alive

Define a function to run "Strategy 2" on a given maze. This strategy identifies the shortest path from the start to the goal location in the given maze everytime before it is the agent's turn to move. The agent moves along this path and the fire is advanced every time after the agent moves. If the agent reaches the goal, True is returned. Otherwise, False is returned.

In [ ]:
def second_Strategy(maze):
    m = maze
    agent = (0,0)
    shortestPath = None
    while True: #Loop exit conditions: Agent reaches goal or agent gets burned
        result = shortestPathSearch(m, agent) #recalculate the shortest path from the agent to the goal at agent's turn
        if result != None:
            shortestPath = result[0]
        else:
            return False
        if shortestPath != None: #if a shortest path was found, move the agent in that direction
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1): #returns true if the agent reaches the goal, otherwise continue
            return True
        m.iterateFire() #advances the fire immediately after the agent's turn
        if m.board[agent] == 2: #returns false if the fire reaches the agent, otherwise continue
            return False

Define a function to run "Strategy 3" on a given maze. This strategy identifies the shortest weighted path from the start to the goal location in the given maze using the adjacentFireHeuristic (for custom weight assignments) everytime before it is the agent's turn to move. The agent moves along this path and the fire is advanced every time after the agent moves. If the agent reaches the goal, True is returned. Otherwise, False is returned.

In [ ]:
def third_Strategy(maze):
    m = maze
    #m.startFire()
    agent = (0,0)
    shortestPath = None
    while True: #Loop exit conditions: Agent reaches goal or agent gets burned
        result = shortestPathSearch(m, agent, heuristicFunction=adjacentFireHeuristic) #recalculate the shortest weighted path from the agent to the goal using the adjacentFireHeuristic
        if result != None:
            shortestPath = result[0]
        else:
            return False
        if shortestPath != None: #if a shortest path exists, advance the agent one step along the path
            agent = shortestPath[1]
        if agent == (m.dim-1, m.dim-1): #if the agent reached the goal, return True
            return True
        m.iterateFire() #advances the fire immediately after the agent's turn
        if m.board[agent] == 2: #if the agent's current cell is now on fire, return False
            return False

Define a heuristic that assigns weight k+1 to a given cell where k is the number of adjacent cells that are on fire.

In [ ]:
def adjacentFireHeuristic(cell, maze, visited):
    cellRow = cell.coords[0]
    cellCol = cell.coords[1]
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)] #identify all potential neighbors to current cell
    weight = 1
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != 2): #if the neighbor is out-of-bounds or not on fire, ignore it
            continue
        weight += 1 #add 1 to weight for each neighbor of the current cell that is on fire
    return weight

In [ ]:
def testStrategy(strategy, dim, p=0.3): # p is set according to project document, strategy must be function name to test
    #dim = 10
    q = 0.1 # start testing each strategy with initial flammability of 0.1
    q_values = [] # list to keep track of the different flammability rates (for plotting later)
    avgSuccesses = [] # list to keep track of the average success rates (for plotting later)
    
    while q < 1: # run strategy for each test value of q (flammability rate)
        run_count = 0 # counter for number of tests run
        success_count = 0 # counter for number of successes
        fail_count = 0 # counter for number of failures
        
        testMaze = Maze(dim, p, q) # generate a maze with given dimension, obstacle probability, and flammability rate
        while(testMaze.isSolvable() == False): # generate a new maze if the maze is not solvable
            testMaze = Maze(dim, p, q)
        maze = copy.deepcopy(testMaze) # create deep copy of inital maze to run the testing
        
        while run_count < 100: 
            maze.startFire() # start fire on created maze
            result = strategy(maze) # run given strategy on the maze and record the result
            if result == True: # if agent survives, increment number of successes for this given q
                success_count +=1
            elif result == None:
                continue
            else:
                fail_count +=1
            run_count += 1
            maze = copy.deepcopy(testMaze) # reset the maze to not include the fires from testing the previous maze
        
        successRate = success_count / (success_count + fail_count) # calculate success rate given all of the generated mazes for given q
        q_values.append(q) 
        avgSuccesses.append(successRate)
        q += 0.05 # increment the flammability rate to test again
        
    q_values_array = np.asarray(q_values) # convert list to array to plot
    avgSuccessRate = np.asarray(avgSuccesses) # convert list to array to plot
    
    plt.figure() # plot the average success rate vs. flammability rate for given strategy 
    plt.plot(q_values_array, avgSuccessRate, 'o')
    plt.xlabel("Flammability Rate")
    plt.ylabel("Average Success Rate")
    plt.title(strategy.__name__)
    plt.show()